In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
from sklearn.metrics import accuracy_score
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import normalize
from sklearn.utils import check_random_state
from tabpfn import TabPFNClassifier

# Fig 1 - non-monotone and asymmetric (32 ensembles) - DONE

# Fig 2 - vary n_ensembles, inverse distance attention - DONE

# Repeat red sample (equivalent to sample weights) - DONE

# Repeat both red and green samples - DONE

# Bladderbatch - sandbox/condo-adapter/paper - DONE

# MNIST, CIFAR- done

In [ ]:
classifier = TabPFNClassifier(device='cpu', n_estimators=1)
X_train = np.array([[-1.], [1.]])
classifier.fit(X_train, np.array(['red', 'green']))
x_eval = np.linspace(-2, 2, 100)
#y_eval, p_eval = classifier.predict(x_eval.reshape(-1, 1), return_winning_probability=True)
y_eval = classifier.predict(x_eval.reshape(-1, 1))
p_eval = classifier.predict_proba(x_eval.reshape(-1, 1)).max(axis=1)
py0 = (y_eval == 'red') * p_eval + (y_eval == 'green') * (1 - p_eval);
py1 = (y_eval == 'green') * p_eval + (y_eval == 'red') * (1 - p_eval);
fig = plt.figure(figsize=(3.5,2), dpi=150);
plt.plot(x_eval, py0, color='red', label='P(Y=red)');
plt.plot(x_eval, py1, color='green', label='P(Y=green)');
plt.plot(x_eval, py1 >= py0, linewidth=1, label="$\hat{Y}$");
plt.xticks([-1., 0., 1.]);
plt.yticks([0., 0.5, 1.]);
plt.gca().get_xticklabels()[0].set_color("red")
plt.gca().get_xticklabels()[2].set_color("green")
plt.xlim(-2, 2);
plt.grid(True);
plt.legend(loc="lower right", labelspacing=0.1, handlelength=0.8, handletextpad=0.35, borderpad=0.25, borderaxespad=0.0);
plt.xlabel("X");
plt.tight_layout();
plt.savefig("/Users/calvinm/sandbox/tabpfn-eval/v2-plusminus1-nonmonotone.pdf");
#sqdists = torch.cdist(torch.from_numpy(x_eval.reshape(-1, 1)), torch.from_numpy(X_train)) ** 0.5
#attn = torch.softmax(1/ (1e-2 + sqdists), axis=1).detach().numpy()
#attn = torch.from_numpy(x_eval.reshape(-1, 1)) @ torch.from_numpy(X_train).T
#attn = torch.softmax(attn, axis=1).detach().numpy()
#plt.plot(x_eval, attn[:, 0], color='orange');
#plt.plot(x_eval, attn[:, 1], color='lime');

In [ ]:
fig = plt.figure(figsize=(5,2), dpi=150);
red_lines = []
tile1_lines = []
for nens in [1, 6, 32]:
    classifier = TabPFNClassifier(device='cpu', n_estimators=nens)
    X_train = np.array([[-1.], [1.]])
    classifier.fit(X_train, np.array(['red', 'green']))
    x_eval = np.linspace(-2, 2, 100)
    #y_eval, p_eval = classifier.predict(x_eval.reshape(-1, 1), return_winning_probability=True)
    y_eval = classifier.predict(x_eval.reshape(-1, 1))
    p_eval = classifier.predict_proba(x_eval.reshape(-1, 1)).max(axis=1)
    py0 = (y_eval == 'red') * p_eval + (y_eval == 'green') * (1 - p_eval);
    py1 = (y_eval == 'green') * p_eval + (y_eval == 'red') * (1 - p_eval);
    lr, = plt.plot(x_eval, py0, color='red', label='P(Y=red)', alpha=1/np.power(nens, 0.5), linewidth=np.power(nens, 0.5));
    lg, = plt.plot(x_eval, py1, color='green', label='P(Y=green)', alpha=1/np.power(nens, 0.5), linewidth=np.power(nens, 0.5));
    red_lines.append(lr)
    if nens == 1:
        tile1_lines.append(lr)
        tile1_lines.append(lg)
        
sqdists = torch.cdist(torch.from_numpy(x_eval.reshape(-1, 1)), torch.from_numpy(X_train)) ** 0.5
attn = torch.softmax(1/ (1.*(1e-3 + sqdists)), axis=1).detach().numpy()
plt.plot(x_eval, attn[:, 0], color='orange');
plt.plot(x_eval, attn[:, 1], color='lime');
plt.xticks([-1., 0., 1.]);
plt.yticks([0., 0.5, 1.]);
plt.gca().get_xticklabels()[0].set_color("red")
plt.gca().get_xticklabels()[2].set_color("green")
plt.xlim(-2, 2);
plt.grid(True);
legend1 = plt.legend(red_lines, [1, 6, 32], bbox_to_anchor=(1.01, 1.03), title="# ensembles")
plt.gca().add_artist(legend1, )
legend2 = plt.legend(tile1_lines, ["P(red)", "P(green)"], bbox_to_anchor=(1.0, 0.3))
plt.gca().add_artist(legend2, )
plt.xlabel("X");
plt.tight_layout(rect=(0, 0, 0.7, 1));
plt.savefig("/Users/calvinm/sandbox/tabpfn-eval/v2-plusminus1-ensembles.png");


In [ ]:
fig = plt.figure(figsize=(5,2), dpi=150);
red_lines = []
tile1_lines = []
for nens in [1, 32]:
    classifier = TabPFNClassifier(device='cpu', n_estimators=nens)
    X_train = np.array([[-1.], [1.]])
    classifier.fit(X_train, np.array(['red', 'green']))
    x_eval = np.linspace(-2, 2, 100)
    X_eval = x_eval.reshape(-1, 1)
    y_eval = classifier.predict(X_eval)
    p_eval = classifier.predict_proba(X_eval).max(axis=1)
    py0 = (y_eval == 'red') * p_eval + (y_eval == 'green') * (1 - p_eval);
    py1 = (y_eval == 'green') * p_eval + (y_eval == 'red') * (1 - p_eval);
    lr, = plt.plot(x_eval, py0, color='red', label='P(Y=red)', alpha=1/np.power(nens, 0.5), linewidth=np.power(nens, 0.5));
    lg, = plt.plot(x_eval, py1, color='green', label='P(Y=green)', alpha=1/np.power(nens, 0.5), linewidth=np.power(nens, 0.5));
    red_lines.append(lr)
    if nens == 1:
        tile1_lines.append(lr)
        tile1_lines.append(lg)
        
sqdists = torch.cdist(torch.from_numpy(x_eval.reshape(-1, 1)), torch.from_numpy(X_train)) ** 0.5
attn = torch.softmax(1/ (1.*(1e-3 + sqdists)), axis=1).detach().numpy()
plt.plot(x_eval, attn[:, 0], color='orange');
plt.plot(x_eval, attn[:, 1], color='lime');
plt.xticks([-1., 0., 1.]);
plt.yticks([0., 0.5, 1.]);
plt.gca().get_xticklabels()[0].set_color("red")
plt.gca().get_xticklabels()[2].set_color("green")
plt.xlim(-2, 2);
plt.grid(True);
legend1 = plt.legend(red_lines, [1, 32], bbox_to_anchor=(1.01, 1.03), title="# ensembles")
plt.gca().add_artist(legend1, )
legend2 = plt.legend(tile1_lines, ["P(red)", "P(green)"], bbox_to_anchor=(1.0, 0.3))
plt.gca().add_artist(legend2, )
plt.xlabel("X");
plt.tight_layout(rect=(0, 0, 0.7, 1));
plt.savefig("/Users/calvinm/sandbox/tabpfn-eval/v2-plusminus1-ensembles2.png");

In [ ]:
# Repeat features, without ensembling
classifier = TabPFNClassifier(device='cpu', n_estimators=1)

red_lines = []
tile1_lines = []
tiles = [1, 4, 16, 64]
tilealphas = 1/np.sqrt(np.array(tiles))
fig = plt.figure(figsize=(5,2), dpi=150);

for tix, tile in enumerate(tiles):
    X_train = np.array([[1.]*tile, [-1.]*tile])
    #X_train = normalize(X_train)
    classifier.fit(X_train, np.array(['green', 'red']))
    x_eval = np.linspace(-2., 2., 100)
    X_eval = np.tile(x_eval.reshape(-1, 1), [1, tile])
    #X_eval = normalize(X_eval)
    #y_eval, p_eval = classifier.predict(X_eval, return_winning_probability=True)
    y_eval = classifier.predict(X_eval)
    p_eval = classifier.predict_proba(X_eval).max(axis=1)
    py0 = (y_eval == 'red') * p_eval + (y_eval == 'green') * (1 - p_eval);
    py1 = (y_eval == 'green') * p_eval + (y_eval == 'red') * (1 - p_eval);
    lr, = plt.plot(x_eval, py0, color='red', label='P(Y=red)', alpha=tilealphas[tix], linewidth=np.power(tile, 0.5));
    lg, = plt.plot(x_eval, py1, color='green', label='P(Y=green)', alpha=tilealphas[tix], linewidth=np.power(tile, 0.5));
    red_lines.append(lr)
    if tix == 1:
        tile1_lines.append(lr)
        tile1_lines.append(lg)
plt.xticks([-1., 0., 1.]);
plt.yticks([0., 0.5, 1.]);
plt.gca().get_xticklabels()[0].set_color("red")
plt.gca().get_xticklabels()[2].set_color("green")
plt.xlim(-2, 2);
plt.grid(True);
legend1 = plt.legend(red_lines, tiles, bbox_to_anchor=(1, 1.05), title="# repeated features")
plt.legend(tile1_lines, ["P(red)", "P(green)"], loc="lower center")
plt.gca().add_artist(legend1, )
legend2 = plt.legend(tile1_lines, ["P(red)", "P(green)"], bbox_to_anchor=(1.07, 0.18))
plt.gca().add_artist(legend2, )
plt.xlabel("X");
plt.tight_layout(rect=(0, 0, 0.7, 1));
plt.savefig("/Users/calvinm/sandbox/tabpfn-eval/v2-plusminus1-repeats.png");

In [ ]:
# Repeat features, with ensembling
classifier = TabPFNClassifier(device='cpu', n_estimators=32)

red_lines = []
tile1_lines = []
tilealphas = 1/np.sqrt(np.array(tiles))
fig = plt.figure(figsize=(5,2), dpi=150);tiles = [1, 4, 16, 64]

for tix, tile in enumerate(tiles):
    X_train = np.array([[1.]*tile, [-1.]*tile])
    #X_train = normalize(X_train)
    classifier.fit(X_train, np.array(['green', 'red']))
    x_eval = np.linspace(-2., 2., 100)
    X_eval = np.tile(x_eval.reshape(-1, 1), [1, tile])
    #X_eval = normalize(X_eval)
    #y_eval, p_eval = classifier.predict(X_eval, return_winning_probability=True)
    y_eval = classifier.predict(X_eval)
    p_eval = classifier.predict_proba(X_eval).max(axis=1)
    py0 = (y_eval == 'red') * p_eval + (y_eval == 'green') * (1 - p_eval);
    py1 = (y_eval == 'green') * p_eval + (y_eval == 'red') * (1 - p_eval);
    lr, = plt.plot(x_eval, py0, color='red', label='P(Y=red)', alpha=tilealphas[tix], linewidth=np.power(tile, 0.5));
    lg, = plt.plot(x_eval, py1, color='green', label='P(Y=green)', alpha=tilealphas[tix], linewidth=np.power(tile, 0.5));
    red_lines.append(lr)
    if tix == 1:
        tile1_lines.append(lr)
        tile1_lines.append(lg)
plt.xticks([-1., 0., 1.]);
plt.yticks([0., 0.5, 1.]);
plt.gca().get_xticklabels()[0].set_color("red")
plt.gca().get_xticklabels()[2].set_color("green")
plt.xlim(-2, 2);
plt.grid(True);
legend1 = plt.legend(red_lines, tiles, bbox_to_anchor=(1, 1.05), title="# repeated features")
plt.legend(tile1_lines, ["P(red)", "P(green)"], loc="lower center")
plt.gca().add_artist(legend1, )
legend2 = plt.legend(tile1_lines, ["P(red)", "P(green)"], bbox_to_anchor=(1.07, 0.18))
plt.gca().add_artist(legend2, )
plt.xlabel("X");
plt.tight_layout(rect=(0, 0, 0.7, 1));
plt.savefig("/Users/calvinm/sandbox/tabpfn-eval/v2-plusminus1-repeats-w32.pdf");

In [ ]:
# Repeat red sample w 32 ensembles
classifier = TabPFNClassifier(device='cpu', n_estimators=32)

red_lines = []
tile1_lines = []
tilealphas = 1/np.sqrt(np.array(tiles))
fig = plt.figure(figsize=(5,2), dpi=150);
tiles = [1, 4, 16]

for tix, tile in enumerate(tiles):
    X_train = np.array([1.]+[-1.]*tile).reshape(-1, 1)
    y_train = np.array(['green']+['red']*tile)
    classifier.fit(X_train, y_train)
    x_eval = np.linspace(-2., 2., 100)
    X_eval = x_eval.reshape(-1, 1)
    #y_eval, p_eval = classifier.predict(X_eval, return_winning_probability=True)
    y_eval = classifier.predict(X_eval)
    p_eval = classifier.predict_proba(X_eval).max(axis=1)
    py0 = (y_eval == 'red') * p_eval + (y_eval == 'green') * (1 - p_eval);
    py1 = (y_eval == 'green') * p_eval + (y_eval == 'red') * (1 - p_eval);
    lr, = plt.plot(x_eval, py0, color='red', label='P(Y=red)', alpha=tilealphas[tix], linewidth=np.power(tile, 0.5));
    lg, = plt.plot(x_eval, py1, color='green', label='P(Y=green)', alpha=tilealphas[tix], linewidth=np.power(tile, 0.5));
    #plt.plot(x_eval, y_eval == 'green', 'k');
    red_lines.append(lr)
    if tix == 1:
        tile1_lines.append(lr)
        tile1_lines.append(lg)
plt.xticks([-1., 0., 1.]);
plt.yticks([0., 0.5, 1.]);
plt.gca().get_xticklabels()[0].set_color("red")
plt.gca().get_xticklabels()[2].set_color("green")
plt.xlim(-2, 2);
plt.grid(True);
legend1 = plt.legend(red_lines, tiles, bbox_to_anchor=(1, 1.05), title="# red samples")
plt.legend(tile1_lines, ["P(red)", "P(green)"], loc="lower center")
plt.gca().add_artist(legend1, )
legend2 = plt.legend(tile1_lines, ["P(red)", "P(green)"], bbox_to_anchor=(1.0, 0.25))
plt.gca().add_artist(legend2, )
plt.xlabel("X");
plt.tight_layout(rect=(0, 0, 0.7, 1));
#plt.savefig("/Users/calvinm/sandbox/tabpfn-eval/plusminus1-repeatred-w32.pdf");

In [ ]:
# Repeat red sample w no ensembles
classifier = TabPFNClassifier(device='cpu', n_estimators=1)

red_lines = []
tile1_lines = []
tilealphas = 1/np.sqrt(np.array(tiles))
fig = plt.figure(figsize=(5,2), dpi=150);
tiles = [1, 4, 16]

for tix, tile in enumerate(tiles):
    X_train = np.array([1.]+[-1.]*tile).reshape(-1, 1)
    y_train = np.array(['green']+['red']*tile)
    classifier.fit(X_train, y_train)
    x_eval = np.linspace(-2., 2., 100)
    X_eval = x_eval.reshape(-1, 1)
    #y_eval, p_eval = classifier.predict(X_eval, return_winning_probability=True)
    y_eval = classifier.predict(X_eval)
    p_eval = classifier.predict_proba(X_eval).max(axis=1)
    py0 = (y_eval == 'red') * p_eval + (y_eval == 'green') * (1 - p_eval);
    py1 = (y_eval == 'green') * p_eval + (y_eval == 'red') * (1 - p_eval);
    lr, = plt.plot(x_eval, py0, color='red', label='P(Y=red)', alpha=tilealphas[tix], linewidth=np.power(tile, 0.5));
    lg, = plt.plot(x_eval, py1, color='green', label='P(Y=green)', alpha=tilealphas[tix], linewidth=np.power(tile, 0.5));
    #plt.plot(x_eval, y_eval == 'green', 'k');
    red_lines.append(lr)
    if tix == 1:
        tile1_lines.append(lr)
        tile1_lines.append(lg)
plt.xticks([-1., 0., 1.]);
plt.yticks([0., 0.5, 1.]);
plt.gca().get_xticklabels()[0].set_color("red")
plt.gca().get_xticklabels()[2].set_color("green")
plt.xlim(-2, 2);
plt.grid(True);
legend1 = plt.legend(red_lines, tiles, bbox_to_anchor=(1, 1.05), title="# red samples")
plt.legend(tile1_lines, ["P(red)", "P(green)"], loc="lower center")
plt.gca().add_artist(legend1, )
legend2 = plt.legend(tile1_lines, ["P(red)", "P(green)"], bbox_to_anchor=(1.0, 0.25))
plt.gca().add_artist(legend2, )
plt.xlabel("X");
plt.tight_layout(rect=(0, 0, 0.7, 1));
plt.savefig("/Users/calvinm/sandbox/tabpfn-eval/v2-plusminus1-repeatred.pdf");

In [ ]:
# Repeat red and green sample w 32 ensembles
classifier = TabPFNClassifier(device='cpu', n_estimators=32)

red_lines = []
tile1_lines = []
tilealphas = 1/np.sqrt(np.array(tiles))
fig = plt.figure(figsize=(5,2), dpi=150);
tiles = [1, 4, 16]

for tix, tile in enumerate(tiles):
    X_train = np.array([1.]*tile+[-1.]*tile).reshape(-1, 1)
    y_train = np.array(['green']*tile+['red']*tile)
    classifier.fit(X_train, y_train)
    x_eval = np.linspace(-2., 2., 100)
    X_eval = x_eval.reshape(-1, 1)
    #y_eval, p_eval = classifier.predict(X_eval, return_winning_probability=True)
    y_eval = classifier.predict(X_eval)
    p_eval = classifier.predict_proba(X_eval).max(axis=1)
    py0 = (y_eval == 'red') * p_eval + (y_eval == 'green') * (1 - p_eval);
    py1 = (y_eval == 'green') * p_eval + (y_eval == 'red') * (1 - p_eval);
    lr, = plt.plot(x_eval, py0, color='red', label='P(Y=red)', alpha=tilealphas[tix], linewidth=np.power(tile, 0.5));
    lg, = plt.plot(x_eval, py1, color='green', label='P(Y=green)', alpha=tilealphas[tix], linewidth=np.power(tile, 0.5));
    #plt.plot(x_eval, y_eval == 'green', 'k');
    red_lines.append(lr)
    if tix == 1:
        tile1_lines.append(lr)
        tile1_lines.append(lg)
plt.xticks([-1., 0., 1.]);
plt.yticks([0., 0.5, 1.]);
plt.gca().get_xticklabels()[0].set_color("red")
plt.gca().get_xticklabels()[2].set_color("green")
plt.xlim(-2, 2);
plt.grid(True);
legend1 = plt.legend(red_lines, tiles, bbox_to_anchor=(1, 1.05), title="# sample copies")
plt.legend(tile1_lines, ["P(red)", "P(green)"], loc="lower center")
plt.gca().add_artist(legend1, )
legend2 = plt.legend(tile1_lines, ["P(red)", "P(green)"], bbox_to_anchor=(1.0, 0.25))
plt.gca().add_artist(legend2, )
plt.xlabel("X");
plt.tight_layout(rect=(0, 0, 0.7, 1));
plt.savefig("/Users/calvinm/sandbox/tabpfn-eval/v2-plusminus1-repeatboth-w32.pdf");

In [ ]:
# Repeat red and green sample w 32 ensembles
classifier = TabPFNClassifier(device='cpu', n_estimators=1)

red_lines = []
tile1_lines = []
tilealphas = 1/np.sqrt(np.array(tiles))
fig = plt.figure(figsize=(5,2), dpi=150);
tiles = [1, 4, 16]

for tix, tile in enumerate(tiles):
    X_train = np.array([1.]*tile+[-1.]*tile).reshape(-1, 1)
    y_train = np.array(['green']*tile+['red']*tile)
    classifier.fit(X_train, y_train)
    x_eval = np.linspace(-2., 2., 100)
    X_eval = x_eval.reshape(-1, 1)
    #y_eval, p_eval = classifier.predict(X_eval, return_winning_probability=True)
    y_eval = classifier.predict(X_eval)
    p_eval = classifier.predict_proba(X_eval).max(axis=1)
    py0 = (y_eval == 'red') * p_eval + (y_eval == 'green') * (1 - p_eval);
    py1 = (y_eval == 'green') * p_eval + (y_eval == 'red') * (1 - p_eval);
    lr, = plt.plot(x_eval, py0, color='red', label='P(Y=red)', alpha=tilealphas[tix], linewidth=np.power(tile, 0.5));
    lg, = plt.plot(x_eval, py1, color='green', label='P(Y=green)', alpha=tilealphas[tix], linewidth=np.power(tile, 0.5));
    #plt.plot(x_eval, y_eval == 'green', 'k');
    red_lines.append(lr)
    if tix == 1:
        tile1_lines.append(lr)
        tile1_lines.append(lg)
plt.xticks([-1., 0., 1.]);
plt.yticks([0., 0.5, 1.]);
plt.gca().get_xticklabels()[0].set_color("red")
plt.gca().get_xticklabels()[2].set_color("green")
plt.xlim(-2, 2);
plt.grid(True);
legend1 = plt.legend(red_lines, tiles, bbox_to_anchor=(1, 1.05), title="# sample copies")
plt.legend(tile1_lines, ["P(red)", "P(green)"], loc="lower center")
plt.gca().add_artist(legend1, )
legend2 = plt.legend(tile1_lines, ["P(red)", "P(green)"], bbox_to_anchor=(1.0, 0.25))
plt.gca().add_artist(legend2, )
plt.xlabel("X");
plt.tight_layout(rect=(0, 0, 0.7, 1));
plt.savefig("/Users/calvinm/sandbox/tabpfn-eval/v2-plusminus1-repeatboth.pdf");

In [ ]:
# Repeat red sample w 32 ensembles
classifier = TabPFNClassifier(device='cpu', n_estimators=32)

red_lines = []
tile1_lines = []
tilealphas = 1/np.sqrt(np.array([1, 6, 32]))
fig = plt.figure(figsize=(5,2), dpi=150);
rng = check_random_state(42)
tile = 3
for nix, nens in enumerate([1, 6, 32]):
    classifier = TabPFNClassifier(device='cpu', n_estimators=nens)
    X_train = np.array([1.]+list(range(-1, -tile-1, -1))).reshape(-1, 1)
    y_train = np.array(['green']+['red']*tile)
    classifier.fit(X_train, y_train)
    x_eval = np.linspace(-5., 2., 100)
    X_eval = x_eval.reshape(-1, 1)
    #y_eval, p_eval = classifier.predict(X_eval, return_winning_probability=True)
    y_eval = classifier.predict(X_eval)
    p_eval = classifier.predict_proba(X_eval).max(axis=1)
    py0 = (y_eval == 'red') * p_eval + (y_eval == 'green') * (1 - p_eval);
    py1 = (y_eval == 'green') * p_eval + (y_eval == 'red') * (1 - p_eval);
    lr, = plt.plot(x_eval, py0, color='red', label='P(Y=red)', alpha=1/np.power(nens, 0.5), linewidth=np.power(nens, 0.5));
    lg, = plt.plot(x_eval, py1, color='green', label='P(Y=green)', alpha=1/np.power(nens, 0.5), linewidth=np.power(nens, 0.5));
    red_lines.append(lr)
    if nix == 1:
        tile1_lines.append(lr)
        tile1_lines.append(lg)
plt.xticks([-5, -4, -3, -2, -1., 0., 1.]);
plt.yticks([0., 0.5, 1.]);
plt.gca().get_xticklabels()[0].set_color("red")
plt.gca().get_xticklabels()[1].set_color("red")
plt.gca().get_xticklabels()[2].set_color("red")
plt.gca().get_xticklabels()[3].set_color("red")
plt.gca().get_xticklabels()[4].set_color("red")
plt.gca().get_xticklabels()[6].set_color("green")
#plt.xlim(-2, 2);
plt.grid(True);
legend1 = plt.legend(red_lines, [1, 6, 32], bbox_to_anchor=(1, 1.05), title="# ensembles")
plt.legend(tile1_lines, ["P(red)", "P(green)"], loc="lower center")
plt.gca().add_artist(legend1, )
legend2 = plt.legend(tile1_lines, ["P(red)", "P(green)"], bbox_to_anchor=(1.0, 0.25))
plt.gca().add_artist(legend2, )
plt.xlabel("X");
plt.tight_layout(rect=(0, 0, 0.7, 1));
plt.savefig("/Users/calvinm/sandbox/tabpfn-eval/v2-plusminus1-repeatred-w32-range.pdf");

In [ ]:
# Periodic behavior
classifier = TabPFNClassifier(device='cpu', n_estimators=32)

red_lines = []
tile1_lines = []
tilealphas = 1/np.sqrt(np.array([1, 6, 32]))
fig = plt.figure(figsize=(5,2), dpi=150);
rng = check_random_state(42)
tile = 3
for nix, nens in enumerate([1, 6, 32]):
    classifier = TabPFNClassifier(device='cpu', n_estimators=nens)
    X_train = np.array(list(range(-5, 1))).reshape(-1, 1)
    y_train = np.array(['green','red']*3)
    classifier.fit(X_train, y_train)
    x_eval = np.linspace(-5., 5., 100)
    X_eval = x_eval.reshape(-1, 1)
    #y_eval, p_eval = classifier.predict(X_eval, return_winning_probability=True)
    y_eval = classifier.predict(X_eval)
    p_eval = classifier.predict_proba(X_eval).max(axis=1)
    py0 = (y_eval == 'red') * p_eval + (y_eval == 'green') * (1 - p_eval);
    py1 = (y_eval == 'green') * p_eval + (y_eval == 'red') * (1 - p_eval);
    lr, = plt.plot(x_eval, py0, color='red', label='P(Y=red)', alpha=1/np.power(nens, 0.5), linewidth=np.power(nens, 0.5));
    lg, = plt.plot(x_eval, py1, color='green', label='P(Y=green)', alpha=1/np.power(nens, 0.5), linewidth=np.power(nens, 0.5));
    red_lines.append(lr)
    if nix == 1:
        tile1_lines.append(lr)
        tile1_lines.append(lg)
plt.xticks(list(range(-5, 6)));
plt.yticks([0., 0.5, 1.]);
for curx, cury in zip(list(range(-5, 1)), ['green','red']*3):
    plt.gca().get_xticklabels()[curx+5].set_color(cury)
plt.xlim(-5, 5);
plt.grid(True);
legend1 = plt.legend(red_lines, [1, 6, 32], bbox_to_anchor=(1, 1.05), title="# ensembles")
plt.legend(tile1_lines, ["P(red)", "P(green)"], loc="lower center")
plt.gca().add_artist(legend1, )
legend2 = plt.legend(tile1_lines, ["P(red)", "P(green)"], bbox_to_anchor=(1.37, 0.25))
plt.gca().add_artist(legend2, )
plt.xlabel("X");
plt.tight_layout(rect=(0, 0, 0.8, 1));
plt.savefig("/Users/calvinm/sandbox/tabpfn-eval/v2-plusminus1-repeatred-w32-range.pdf");

In [ ]:
# Periodic behavior
classifier = TabPFNClassifier(device='cpu', n_estimators=32)

red_lines = []
tile1_lines = []
tilealphas = 1/np.sqrt(np.array([1, 6, 32]))
fig = plt.figure(figsize=(5,2), dpi=150);
rng = check_random_state(42)
tile = 3
for nix, nens in enumerate([1, 6, 32]):
    classifier = TabPFNClassifier(device='cpu', n_estimators=nens)
    X_train = np.array(list(range(-3, 1))).reshape(-1, 1)
    y_train = np.array(['green','red']*2)
    classifier.fit(X_train, y_train)
    x_eval = np.linspace(-5., 5., 100)
    X_eval = x_eval.reshape(-1, 1)
    #y_eval, p_eval = classifier.predict(X_eval, return_winning_probability=True)
    y_eval = classifier.predict(X_eval)
    p_eval = classifier.predict_proba(X_eval).max(axis=1)
    py0 = (y_eval == 'red') * p_eval + (y_eval == 'green') * (1 - p_eval);
    py1 = (y_eval == 'green') * p_eval + (y_eval == 'red') * (1 - p_eval);
    lr, = plt.plot(x_eval, py0, color='red', label='P(Y=red)', alpha=1/np.power(nens, 0.5), linewidth=np.power(nens, 0.5));
    lg, = plt.plot(x_eval, py1, color='green', label='P(Y=green)', alpha=1/np.power(nens, 0.5), linewidth=np.power(nens, 0.5));
    red_lines.append(lr)
    if nix == 1:
        tile1_lines.append(lr)
        tile1_lines.append(lg)
plt.xticks(list(range(-5, 6)));
plt.yticks([0., 0.5, 1.]);
for curx, cury in zip(list(range(-3, 1)), ['green','red']*3):
    plt.gca().get_xticklabels()[curx+5].set_color(cury)
plt.xlim(-5, 5);
plt.grid(True);
legend1 = plt.legend(red_lines, [1, 6, 32], bbox_to_anchor=(1, 1.05), title="# ensembles")
plt.legend(tile1_lines, ["P(red)", "P(green)"], loc="lower center")
plt.gca().add_artist(legend1, )
legend2 = plt.legend(tile1_lines, ["P(red)", "P(green)"], bbox_to_anchor=(1.37, 0.25))
plt.gca().add_artist(legend2, )
plt.xlabel("X");
plt.tight_layout(rect=(0, 0, 0.8, 1));
plt.savefig("/Users/calvinm/sandbox/tabpfn-eval/v2-plusminus1-repeatred-w32-shortrange.pdf");

In [ ]:
# Periodic behavior
red_lines = []
tile1_lines = []
tilealphas = 1/np.sqrt(np.array([1, 6, 32]))
fig = plt.figure(figsize=(5,1.5), dpi=150);
rng = check_random_state(42)
tile = 3
for nix, nens in enumerate([1]):
    classifier = TabPFNClassifier(device='cpu', n_estimators=nens)
    X_train = np.array(list(range(-3, 1))).reshape(-1, 1)
    y_train = np.array(['green','red']*2)
    classifier.fit(X_train, y_train)
    x_eval = np.linspace(-5., 5., 100)
    X_eval = x_eval.reshape(-1, 1)
    #y_eval, p_eval = classifier.predict(X_eval, return_winning_probability=True)
    y_eval = classifier.predict(X_eval)
    p_eval = classifier.predict_proba(X_eval).max(axis=1)
    py0 = (y_eval == 'red') * p_eval + (y_eval == 'green') * (1 - p_eval);
    py1 = (y_eval == 'green') * p_eval + (y_eval == 'red') * (1 - p_eval);
    lr, = plt.plot(x_eval, py0, color='red', label='P(Y=red)', alpha=1/np.power(nens, 0.5), linewidth=np.power(nens, 0.5));
    lg, = plt.plot(x_eval, py1, color='green', label='P(Y=green)', alpha=1/np.power(nens, 0.5), linewidth=np.power(nens, 0.5));
    red_lines.append(lr)
    if nix == 0:
        tile1_lines.append(lr)
        tile1_lines.append(lg)
plt.xticks(list(range(-5, 6)));
plt.yticks([0., 0.5, 1.]);
for curx, cury in zip(list(range(-3, 1)), ['green','red']*3):
    plt.gca().get_xticklabels()[curx+5].set_color(cury)
plt.xlim(-5, 5);
plt.grid(True);
#legend1 = plt.legend(red_lines, [1, 6, 32], bbox_to_anchor=(1, 1.05), title="# ensembles")
#plt.legend(tile1_lines, ["P(red)", "P(green)"], loc="lower center")
#plt.gca().add_artist(legend1, )
legend2 = plt.legend(tile1_lines, ["P(red)", "P(green)"], bbox_to_anchor=(1.37, 0.9))
plt.gca().add_artist(legend2, )
plt.xlabel("X");
plt.tight_layout(rect=(0, 0, 0.8, 1));
plt.savefig("/Users/calvinm/sandbox/tabpfn-eval/v2-plusminus1-periodic-2pair.pdf");

In [ ]:
# Periodic behavior
red_lines = []
tile1_lines = []
tilealphas = 1/np.sqrt(np.array([1, 6, 32]))
fig = plt.figure(figsize=(5,1.5), dpi=150);
rng = check_random_state(42)
tile = 3
for nix, nens in enumerate([1]):
    classifier = TabPFNClassifier(device='cpu', n_estimators=nens)
    X_train = np.array(list(range(-3, 3))).reshape(-1, 1)
    y_train = np.array(['green','red']*3)
    classifier.fit(X_train, y_train)
    x_eval = np.linspace(-6., 5., 100)
    X_eval = x_eval.reshape(-1, 1)
    #y_eval, p_eval = classifier.predict(X_eval, return_winning_probability=True)
    y_eval = classifier.predict(X_eval)
    p_eval = classifier.predict_proba(X_eval).max(axis=1)
    py0 = (y_eval == 'red') * p_eval + (y_eval == 'green') * (1 - p_eval);
    py1 = (y_eval == 'green') * p_eval + (y_eval == 'red') * (1 - p_eval);
    lr, = plt.plot(x_eval, py0, color='red', label='P(Y=red)', alpha=1/np.power(nens, 0.5), linewidth=np.power(nens, 0.5));
    lg, = plt.plot(x_eval, py1, color='green', label='P(Y=green)', alpha=1/np.power(nens, 0.5), linewidth=np.power(nens, 0.5));
    red_lines.append(lr)
    if nix == 0:
        tile1_lines.append(lr)
        tile1_lines.append(lg)
plt.xticks(list(range(-5, 6)));
plt.yticks([0., 0.5, 1.]);
for curx, cury in zip(list(range(-3, 3)), ['green','red']*3):
    plt.gca().get_xticklabels()[curx+5].set_color(cury)
plt.xlim(-5, 5);
plt.grid(True);
#legend1 = plt.legend(red_lines, [1, 6, 32], bbox_to_anchor=(1, 1.05), title="# ensembles")
#plt.legend(tile1_lines, ["P(red)", "P(green)"], loc="lower center")
#plt.gca().add_artist(legend1, )
legend2 = plt.legend(tile1_lines, ["P(red)", "P(green)"], bbox_to_anchor=(1.37, 0.9))
plt.gca().add_artist(legend2, )
plt.xlabel("X");
plt.tight_layout(rect=(0, 0, 0.8, 1));
plt.savefig("/Users/calvinm/sandbox/tabpfn-eval/v2-plusminus1-periodic-3pair.pdf");

In [ ]:
# Periodic behavior
red_lines = []
tile1_lines = []
tilealphas = 1/np.sqrt(np.array([1, 6, 32]))
fig = plt.figure(figsize=(5,1.5), dpi=150);
rng = check_random_state(42)
tile = 3
for nix, nens in enumerate([1]):
    classifier = TabPFNClassifier(device='cpu', n_estimators=nens)
    X_train = np.array([-3., -2., 1., 2.]).reshape(-1, 1)
    y_train = np.array(['green','red']*2)
    classifier.fit(X_train, y_train)
    x_eval = np.linspace(-5., 5., 100)
    X_eval = x_eval.reshape(-1, 1)
    #y_eval, p_eval = classifier.predict(X_eval, return_winning_probability=True)
    y_eval = classifier.predict(X_eval)
    p_eval = classifier.predict_proba(X_eval).max(axis=1)
    py0 = (y_eval == 'red') * p_eval + (y_eval == 'green') * (1 - p_eval);
    py1 = (y_eval == 'green') * p_eval + (y_eval == 'red') * (1 - p_eval);
    lr, = plt.plot(x_eval, py0, color='red', label='P(Y=red)', alpha=1/np.power(nens, 0.5), linewidth=np.power(nens, 0.5));
    lg, = plt.plot(x_eval, py1, color='green', label='P(Y=green)', alpha=1/np.power(nens, 0.5), linewidth=np.power(nens, 0.5));
    red_lines.append(lr)
    if nix == 0:
        tile1_lines.append(lr)
        tile1_lines.append(lg)
plt.xticks(list(range(-5, 6)));
plt.yticks([0., 0.5, 1.]);
for curx, cury in zip(list(range(-3, -1)), ['green','red']*1):
    plt.gca().get_xticklabels()[curx+5].set_color(cury)
for curx, cury in zip(list(range(1, 3)), ['green','red']*1):
    plt.gca().get_xticklabels()[curx+5].set_color(cury)
plt.xlim(-5, 5);
plt.grid(True);
#legend1 = plt.legend(red_lines, [1, 6, 32], bbox_to_anchor=(1, 1.05), title="# ensembles")
#plt.legend(tile1_lines, ["P(red)", "P(green)"], loc="lower center")
#plt.gca().add_artist(legend1, )
legend2 = plt.legend(tile1_lines, ["P(red)", "P(green)"], bbox_to_anchor=(1.37, 0.9))
plt.gca().add_artist(legend2, )
plt.xlabel("X");
plt.tight_layout(rect=(0, 0, 0.8, 1));
plt.savefig("/Users/calvinm/sandbox/tabpfn-eval/v2-plusminus1-periodic-2skippair.pdf");

In [ ]:
# Periodic behavior
red_lines = []
tile1_lines = []
tilealphas = 1/np.sqrt(np.array([1, 6, 32]))
fig = plt.figure(figsize=(5,1.5), dpi=150);
rng = check_random_state(42)
tile = 3
for nix, nens in enumerate([1]):
    classifier = TabPFNClassifier(device='cpu', n_estimators=nens)
    X_train = np.array([-5., -4., -3., -2., 1., 2., 3., 4.]).reshape(-1, 1)
    y_train = np.array(['green','red']*4)
    classifier.fit(X_train, y_train)
    x_eval = np.linspace(-5., 5., 100)
    X_eval = x_eval.reshape(-1, 1)
    #y_eval, p_eval = classifier.predict(X_eval, return_winning_probability=True)
    y_eval = classifier.predict(X_eval)
    p_eval = classifier.predict_proba(X_eval).max(axis=1)
    py0 = (y_eval == 'red') * p_eval + (y_eval == 'green') * (1 - p_eval);
    py1 = (y_eval == 'green') * p_eval + (y_eval == 'red') * (1 - p_eval);
    lr, = plt.plot(x_eval, py0, color='red', label='P(Y=red)', alpha=1/np.power(nens, 0.5), linewidth=np.power(nens, 0.5));
    lg, = plt.plot(x_eval, py1, color='green', label='P(Y=green)', alpha=1/np.power(nens, 0.5), linewidth=np.power(nens, 0.5));
    red_lines.append(lr)
    if nix == 0:
        tile1_lines.append(lr)
        tile1_lines.append(lg)
plt.xticks(list(range(-5, 6)));
plt.yticks([0., 0.5, 1.]);
for curx, cury in zip(list(range(-3, -1)), ['green','red']*1):
    plt.gca().get_xticklabels()[curx+5].set_color(cury)
for curx, cury in zip(list(range(1, 3)), ['green','red']*1):
    plt.gca().get_xticklabels()[curx+5].set_color(cury)
plt.xlim(-5, 5);
plt.grid(True);
#legend1 = plt.legend(red_lines, [1, 6, 32], bbox_to_anchor=(1, 1.05), title="# ensembles")
#plt.legend(tile1_lines, ["P(red)", "P(green)"], loc="lower center")
#plt.gca().add_artist(legend1, )
legend2 = plt.legend(tile1_lines, ["P(red)", "P(green)"], bbox_to_anchor=(1.37, 0.9))
plt.gca().add_artist(legend2, )
plt.xlabel("X");
plt.tight_layout(rect=(0, 0, 0.8, 1));
plt.savefig("/Users/calvinm/sandbox/tabpfn-eval/v2-plusminus1-periodic-4skippair.pdf");

In [ ]:
classifier = TabPFNClassifier(device='cpu', n_estimators=1)
X_train = np.array([[-1.], [1.]])
classifier.fit(X_train, np.array(['red', 'green']))
x_eval = np.linspace(-2, 2, 100)
X_eval = x_eval.reshape(-1, 1)
y_eval = classifier.predict(X_eval)
p_eval = classifier.predict_proba(X_eval).max(axis=1)
py0 = (y_eval == 'red') * p_eval + (y_eval == 'green') * (1 - p_eval);
py1 = (y_eval == 'green') * p_eval + (y_eval == 'red') * (1 - p_eval);
fig = plt.figure(figsize=(3.5,2), dpi=150);
plt.plot(x_eval, py0, color='red', label='P(Y=red)');
plt.plot(x_eval, py1, color='green', label='P(Y=green)');
plt.plot(x_eval, py1 >= py0, linewidth=1, label="$\hat{Y}$");
plt.xticks([-1., 0., 1.]);
plt.yticks([0., 0.5, 1.]);
plt.gca().get_xticklabels()[0].set_color("red")
plt.gca().get_xticklabels()[2].set_color("green")
plt.xlim(-2, 2);
plt.grid(True);
plt.legend(loc="lower right", labelspacing=0.1, handlelength=0.8, handletextpad=0.35, borderpad=0.25, borderaxespad=0.0);
plt.xlabel("X");
plt.tight_layout();